Vamos a nutrir los datos de los ataques de los tiburones con información climática de los paises que tenemos. Para eso vamos a usar la API del clima que hemos aprendido en la clase invertida:

`url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'`

En este caso os recomendamos que uséis el producto `meteo` para obtener la información climática. Para hacer la llamada a la API necesitamos también las coordenadas de los paises que tenemos en el *dataset*.

Por lo tanto, el objetivo es que saquéis la información del clima de la API para los paises que tenemos. Pero antes de poneros manos a la obra, tenemos muchísimos países y esto puede hacerse eterno. Sacad la información climática solo para los siguientes países, con las siguientes coordenadas:

>Pais, Latitud, Longitud
>
>USA,
>39.7837304,
>-100.445882
>
>Australia,
>-24.7761086,
>134.755
>
>South Africa,
>-28.8166236,
>24.991639
>
>New Zealand,
>-41.5000831,
>172.8344077
>
>Papua New Guinea,
>-5.6816069,
>144.2489081



* Requisitos de este ejercicio de pair programming:
* Deberéis meter toda la información climática en un único dataframe.
* Deberéis hacer la llamada a la API de una sola vez. Es decir, tendréis que iterar por la lista de países y sacar la información del clima para cada uno de ellos.
* Al meter toda la información en un único dataframe tendremos que crear una columna que corresponda con el nombre del país.
* Guardar el dataframe obtenido para usarlo en la siguiente sesión de pair programming.

💡 Pistas 💡:

* Crearos un diccionario donde:
    * Las keys sean los nombres de los paises

    * Los values sean las coordenadas de los paises en formato lista.

* Iterar por el diccionario. Dentro del for haremos la llamada a la API como hemos aprendido en las clases invertidas. Recordamos que para iterar por el diccionario tenemos que seguir la siguiente estructura:
```python 
for key, value in diccionario.items():
    print(key, value)
```
* Crearnos un dataframe vacío donde iremos concatenando los datos obtenidos de la API cada país.

* Guardad el csv ya que mañana lo necesitaremos para hacer el ejercicio de pair programming.

In [11]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [12]:
# Definimos el diccionario
diccionario = {'usa':[39.7837304, -100.445882], 
               'australia':[-24.7761086, 134.755], 
               'south africa':[-28.8166236, 24.991639], 
               'new zealand':[-41.5000831, 172.8344077], 
               'papua new guinea':[-5.6816069, 144.2489081]}

In [13]:
# Tras probar cómo se haría con un solo país
# Hacemos el bucle for

def extracciones_paises(diccionario):
    # Creamos un df vacío
    df_all= pd.DataFrame()
    
    for key, value in diccionario.items():
        # Extraemos la latitud y la longitud
        lat = value[0]
        lon = value[1]
        pais = key
        producto = 'meteo'

        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        # Llamamos a la Api
        response = requests.get(url)
     
        df = pd.json_normalize(response.json()['dataseries'])
        df['pais'] = key
        df['latitud'] = lat
        df['longitud'] = lon
        # Concatenamos cada df extraído al df vacío inicial
        df_all = pd.concat([df_all, df], axis = 0, ignore_index=True)

        if response.status_code == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',response.status_code,' y como razón del código de estado: ',response.reason)
        elif response.status_code == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
        elif response.status_code == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
        
    return df_all
    

In [14]:
df_paises = extracciones_paises(diccionario)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [15]:
# Comprobamos por qué número empiezan y acaban los índices
df_paises.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,3,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 315, 'speed':...",27,-1,5,1010,none,1,0,295,2,usa,39.78373,-100.445882
1,6,3,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 50, 'speed': ...",26,-1,5,1011,none,1,0,25,2,usa,39.78373,-100.445882


In [16]:
# Así es como los queremos
df_paises.tail(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
318,189,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 80, 'speed': ...",26,2,10,1011,none,3,0,80,3,papua new guinea,-5.681607,144.248908
319,192,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",26,2,10,1009,none,3,0,80,3,papua new guinea,-5.681607,144.248908


In [17]:
# Comprobamos la columna países
df_paises['pais'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [18]:
df_paises.shape

(320, 19)

In [19]:
df_paises.dtypes

timepoint              int64
cloudcover             int64
highcloud              int64
midcloud               int64
lowcloud               int64
rh_profile            object
wind_profile          object
temp2m                 int64
lifted_index           int64
rh2m                   int64
msl_pressure           int64
prec_type             object
prec_amount            int64
snow_depth             int64
wind10m.direction     object
wind10m.speed          int64
pais                  object
latitud              float64
longitud             float64
dtype: object

In [20]:
# Vemos que todo está correcto, guardamos en pkl para conservar todos los dtypes
df_paises.to_pickle('datos/output/datos_clima_paises.pkl')